# --IMDB MOVIES TO SQL--
## Data Analysis process
- Import libraries
- Load dataset
- Data overview
- Data Cleaning
- Data to SQL

## *Import libraries*----------------------------------------------------------------------------

In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', None)
#import warnings
#warnings.filterwarnings('ignore')

## *Load Dataset*----------------------------------------------------------------------------

In [167]:
raw_data = pd.read_csv('imdb_raw.csv')

## *Data Overview*----------------------------------------------------------------------------

In [168]:
raw_data.head(4)

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"


As observed, this table contains 9 columns.

Let's see the data types too:

In [169]:
raw_data.dtypes

title           object
year            object
certificate     object
duration        object
genre           object
rating         float64
description     object
stars           object
votes           object
dtype: object

The column "rating" has a dtype of "float64", but the actual values from the csv file only goes 1 digit after de point. We will upgrade this latter, turning the dtype "float64" to "float16" for better performance.

For now, let's take a better look to the "stars" column

In [170]:
raw_data['stars']

0       ['Ralph Macchio, ', 'William Zabka, ', 'Courtn...
1       ['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...
2       ['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...
3       ['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...
4       ['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...
                              ...                        
9952    ['Morgan Taylor Campbell, ', 'Italia Ricci, ',...
9953    ['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...
9954    ['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...
9955    ['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...
9956    ['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...
Name: stars, Length: 9957, dtype: object

This column contain the type of "object", but the if you look closely, it is a list in format of string. Look in the example below

In [171]:
raw_data['stars'][0], type(raw_data['stars'][0])

("['Ralph Macchio, ', 'William Zabka, ', 'Courtney Henggeler, ', 'Xolo Maridueña']",
 str)

Something better can be made with this, creating a table with artists id as primary key, movie id with foreign key and the artist name...

Another weird data is located in the final column of the table, let's look at "votes":

In [172]:
raw_data['votes']

0       177,031
1       199,885
2       501,384
3         9,773
4        15,413
         ...   
9952      3,130
9953    970,067
9954    199,898
9955    439,601
9956      9,786
Name: votes, Length: 9957, dtype: object

This column show to us the number of rating votes to the movie, but the value is "object" and not a int. We need to correct this too

Now, look at the column duration. It shows the time of the movie in minutes, but as we can see, this is an object too, and we could not make any math or mean with this data because of it datatype. So, we need to change this too

In [173]:
raw_data['duration'].head(5)

0     30 min
1     58 min
2     46 min
3    356 min
4     24 min
Name: duration, dtype: object

Finaly, i would like to transform the column "genre", that has a "list" of values separated by comma, and i would like to separate this type of value in another table, similar thing to what we need to do with the "stars"

## *Data Cleaning*----------------------------------------------------------------------------

First, let's make a copy of the raw data before we clean it (it is plausible because this dataset is not large at all)

In [174]:
clean_data = raw_data.copy()

In [175]:
clean_data.head(2)

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"


Now let's start with the easy ones: change de dtype of *rating* and *votes* to float32 and int32. The reason is simply to make the format more compact and improve performance!

In [176]:
clean_data['rating'] = raw_data['rating'].astype(np.float32)
clean_data['rating'].head(3)

0    8.5
1    8.7
2    8.9
Name: rating, dtype: float32

To convert this object to int, we need first to remove the "," from the vote numbers, then we need to choose a value for the null values in "votes" column (i choosed 0), and after we can transform to type int32

In [177]:
clean_data['votes'] = raw_data['votes'].str.replace(',', '')
clean_data['votes'] = clean_data['votes'].fillna(0).astype(np.int32)
clean_data['votes']

0       177031
1       199885
2       501384
3         9773
4        15413
         ...  
9952      3130
9953    970067
9954    199898
9955    439601
9956      9786
Name: votes, Length: 9957, dtype: int32

Let's turn the "duration" column to int32, so we can after make some analysis with this value that would not be possible if it stays in type object

In [178]:
clean_data['duration'] = raw_data['duration'].str.replace(" min", "")
clean_data['duration'] = clean_data['duration'].astype(pd.Int32Dtype())
clean_data['duration']

0        30
1        58
2        46
3       356
4        24
       ... 
9952     45
9953     44
9954     58
9955     44
9956    356
Name: duration, Length: 9957, dtype: Int32

In [179]:
clean_data.dtypes

title           object
year            object
certificate     object
duration         Int32
genre           object
rating         float32
description     object
stars           object
votes            int32
dtype: object

Cool. Looks like that dtypes are all good. Now we need to check some invalid values in this table, maybe some extremely large values or extremely low

In [180]:
clean_data['duration'].max()

np.int32(990)

In [181]:
clean_data.loc[clean_data['duration'] == 990]

,title,year,certificate,duration,genre,rating,description,stars,votes
943,The Vietnam War,(2017),TV-MA,990,"Documentary, History, War",9.1,A comprehensive history of the United States' ...,"['Peter Coyote, ', 'Huy Duc, ', 'James Willban...",24632


The movie with the largest amount of hours is "The Vietnam War", and checking in google, in fact it has this extremely large amount of hours... So nothing to modify here

In [182]:
clean_data.loc[clean_data['duration'] == 1]

,title,year,certificate,duration,genre,rating,description,stars,votes
4155,Horsin' Around,(2014 Video),NaN,1,"Short, Comedy, Music",7.3,The opening title sequence to the spoof 90s si...,[],1035
4766,Piggy Tales,(2014–2018),TV-Y,1,"Animation, Adventure, Comedy",6.8,A cartoon series of green cartoon piggies.,"['Antti Pääkkönen, ', 'Antti Pääkkönen, ', 'Do...",147
5524,Pokémon: Happy Birthday to You!,(2017 TV Short),NaN,1,"Animation, Short",5.0,"Today is your big day, and Ash, Pikachu and al...",[],35
5558,13 Reasons Why: Season 2 Date Announcement Com...,(2018),Not Rated,1,"Short, Drama",7.2,Add a Plot,"['Greg Jardin', '| ', ' Stars:', 'Alisha Bo...",260
6119,Miraculous: Ladybug & Cat Noir: Happy Birthday...,(2017 TV Special),NaN,1,"Animation, Short",7.2,"Break dancing, dragons, balloons and a DJ: Lad...",[],68
6282,Barbie: Happy Birthday to You!,(2017 TV Special),NaN,1,"Animation, Short",4.9,Get ready to make a wish and have the most ama...,"['Ritesh Rajan, ', 'America Young']",16
6287,LEGO Ninjago: Masters of Spinjitzu: Happy Birt...,(2017 TV Special),TV-Y7,1,"Animation, Short, Action",6.6,"LEGO Ninjago heroes Kai, Zane, Jay and Cole wa...",[],15
6423,Luke Cage S2: Fresh to Death,(2018 Video),NaN,1,"Short, Drama",NaN,Luke Cage becomes a hero and celebrity after w...,['Matt T. Sample'],0
6452,Santa Clarita Diet: I Love Bodies,(2018 Video),NaN,1,"Short, Comedy",8.1,Add a Plot,['Matt T. Sample'],9
6985,LEGO Friends: Happy Birthday to You!,(2017 TV Special),NaN,1,"Animation, Short",3.8,Surprise. The gang's all here to celebrate you...,[],7


There we have a problem. This movies show that they have 1 minute of duration, and this is false. We can apply a filter in this whole table that all movies with 4 or less minutes of duration be a NaN value

In [183]:
clean_data.loc[clean_data['duration'] <= 4, "duration"] = pd.NA

In [184]:
clean_data.loc[clean_data['duration'].isnull()]

,title,year,certificate,duration,genre,rating,description,stars,votes
19,Fate: The Winx Saga,(2021– ),TV-MA,<NA>,"Action, Adventure, Drama",6.9,A live-action adaptation of Nickelodeon's Winx...,"['Sadie Soverall, ', 'Abigail Cowen, ', 'Fredd...",43179
67,Wednesday,(2022– ),NaN,<NA>,"Comedy, Family, Fantasy",NaN,"Follows Wednesday Addams' years as a student, ...","['Jenna Ortega, ', 'Christina Ricci, ', 'Cathe...",0
77,The Lørenskog Disappearance,(2021),TV-MA,<NA>,"Documentary, Crime",6.1,What happened to Anne-Elisabeth Hagen? Where i...,[],668
168,Terim,(2022– ),TV-14,<NA>,"Documentary, Biography, Sport",7.0,Legendary manager Fatih Terim recounts his foo...,"['Fatih Terim, ', 'Bülent Timurlenk, ', 'Okan ...",4279
199,Dated and Related,(2022– ),TV-MA,<NA>,Reality-TV,4.3,Pairs of siblings see each other's love life u...,"['Joey Roppo, ', 'Jason Cohen, ', 'Christopher...",379
...,...,...,...,...,...,...,...,...,...
9900,Chocolate,(2019–2020),NaN,<NA>,"Drama, Romance",8.8,Kang chooses what's most important to him. Cha...,"['Lee Hyeong-min', '| ', ' Stars:', 'Ha Ji-...",12
9910,Vikings,(2013–2020),TV-MA,<NA>,"Action, Adventure, Drama",7.5,"After leaving Kattegat, Ivar the Boneless emba...","['Karla Braun', '| ', ' Stars:', 'Alex Høgh...",262
9913,Unstoppable,(2020– ),NaN,<NA>,"Comedy, Drama",8.4,A visit to Vera's vacation house brings back p...,"['Diego Martínez Ulanosky', '| ', ' Stars:'...",41
9920,The Upshaws,(2021– ),TV-14,<NA>,"Comedy, Family",6.4,"When Tasha gets into an accident, Bernie comes...","['Sheldon Epps', '| ', ' Stars:', 'Mike Epp...",97


Time to turn the movies with 0 votes to NaN too

In [185]:
clean_data.loc[clean_data['votes'] == 0, 'votes'] = np.nan

Now the hard part, we need to turn the start column in a separated column from the rest of the table, and make is a "new table". This is for the final process of take it to sql

In [186]:
stars_table = clean_data.pop('stars')

In [187]:
stars_table = pd.DataFrame({'star_id': range(1, len(stars_table) + 1), 'stars':stars_table})

In [188]:
stars_table

,star_id,stars
0,1,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn..."
1,2,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St..."
2,3,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan..."
3,4,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo..."
4,5,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ..."
...,...,...
9952,9953,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',..."
9953,9954,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis..."
9954,9955,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St..."
9955,9956,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ..."


In [190]:
clean_data

,title,year,certificate,duration,genre,rating,description,votes
0,Cobra Kai,(2018– ),TV-14,30,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,177031.0
1,The Crown,(2016– ),TV-MA,58,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,199885.0
2,Better Call Saul,(2015–2022),TV-MA,46,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,501384.0
3,Devil in Ohio,(2022),TV-MA,356,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,9773.0
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,15413.0
...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,3130.0
9953,The Walking Dead,(2010–2022),TV-MA,44,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a com...,970067.0
9954,The Crown,(2016– ),TV-MA,58,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,199898.0
9955,Supernatural,(2005–2020),TV-14,44,"Drama, Fantasy, Horror",8.4,Two brothers follow their father's footsteps a...,439601.0
